In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
!pip install gradio
import gradio as gr

# Load and prepare the dataset
df = pd.read_csv('traffic_accidents_dict new.csv')

# Drop unn ecessary columns
df_clean = df.drop(columns=['Variable', 'Description'])

# Define target and features
X = df_clean.drop(columns=['accidents'])
y = df_clean['accidents']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Define prediction function
def predict_accidents(traffic_fine_amount, traffic_density, traffic_lights,
                      pavement_quality, urban_area, average_speed,
                      rain_intensity, vehicle_count, time_of_day):
    input_data = pd.DataFrame([[
        traffic_fine_amount, traffic_density, traffic_lights,
        pavement_quality, urban_area, average_speed,
        rain_intensity, vehicle_count, time_of_day
    ]], columns=X.columns)

    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)
    return round(prediction[0], 2)

# Create Gradio interface
interface = gr.Interface(
    fn=predict_accidents,
    inputs=[
        gr.Number(label='Traffic Fine Amount'),
        gr.Number(label='Traffic Density'),
        gr.Number(label='Traffic Lights'),
        gr.Number(label='Pavement Quality'),
        gr.Number(label='Urban Area (1 for Urban, 0 for Rural)'),
        gr.Number(label='Average Speed'),
        gr.Number(label='Rain Intensity'),
        gr.Number(label='Vehicle Count'),
        gr.Number(label='Time of Day')
    ],
    outputs='number',
    title='Traffic Accident Prediction',
    description='Predict the number of traffic accidents based on various factors.'
)

interface.launch()